<a href="https://colab.research.google.com/github/hbp5181/Linear-Model-uisng-homolog-survey-data/blob/main/future_learning(sequence_to_numbers).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inverse folding with ESM-IF1

The ESM-IF1 inverse folding model is built for predicting protein sequences from their backbone atom coordinates. We provide examples here 1) to sample sequence designs for a given structure and 2) to score sequences for a given structure.

Trained with 12M protein structures predicted by AlphaFold2, the ESM-IF1 model consists of invariant geometric input processing layers followed by a sequence-to-sequence transformer, and achieves 51% native sequence recovery on structurally held-out backbones. The model is also trained with span masking to tolerate missing backbone coordinates and therefore can predict sequences for partially masked structures.

See [GitHub README](https://github.com/facebookresearch/esm/tree/main/examples/inverse_folding) for the complete user guide, and see our [bioRxiv pre-print](https://doi.org/10.1101/2022.04.10.487779) for more details.

## Environment setup (colab)
This step might take up to 10 minutes the first time.

If using a local jupyter environment, instead of the following, we recommend configuring a conda environment upon first use in command line:
```
conda create -n inverse python=3.9
conda activate inverse
conda install pytorch cudatoolkit=11.3 -c pytorch
conda install pyg -c pyg -c conda-forge
conda install pip
pip install biotite
pip install git+https://github.com/facebookresearch/esm.git
```

Afterwards, `conda activate inverse` to activate this environment before starting `jupyter notebook`.

Below is the setup for colab notebooks:

We recommend using GPU runtimes on colab (Menu bar -> Runtime -> Change runtime type -> Hardware accelerator -> GPU)

In [109]:
# Colab environment setup

# Install the correct version of Pytorch Geometric.
import torch
import os

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric

# Install esm
!pip install -q git+https://github.com/facebookresearch/esm.git

# Install biotite
!pip install -q biotite

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Verify that pytorch-geometric is correctly installed

If the notebook crashes at the import, there is likely an issue with the version of torch_geometric and torch_sparse being incompatible with the torch version.

In [ ]:
## Verify that pytorch-geometric is correctly installed
import torch_geometric
import torch_sparse
from torch_geometric.nn import MessagePassing

## Load model
This steps takes a few minutes for the model to download.

**UPDATE**: It is important to set the model in eval mode through `model = model.eval()` to disable random dropout for optimal performance.

In [ ]:
import esm
model, alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
model = model.eval()

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm_if1_gvp4_t16_142M_UR50.pt" to /root/.cache/torch/hub/checkpoints/esm_if1_gvp4_t16_142M_UR50.pt
/usr/local/lib/python3.10/dist-packages/esm/pretrained.py:215: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


## Load structure from PDB or CIF files

As an example, let's look at Golgi Casein Kinase, the [PDB Molecule of the Month from January 2022](https://pdb101.rcsb.org/motm/265).

Milk is a complex mixture of proteins, fats, and nutrients that provides everything that a growing infant needs. Most of the protein in cow’s milk is casein, whereas human milk has lesser amounts of casein.

The Golgi casein kinase (PDB entry 5YH2) adds phosphates to casein and also to many other types of secreted proteins. It is most active as a complex of two similar types of proteins. Fam20C is the catalytic subunit. It binds to casein and transfers a phosphate from ATP to the protein.

In this example, let's focus on chain C (the catalytic subunit Fam20C).

You may also upload your own CIF or PDB file and specify the chain id.

In [ ]:
!wget https://files.rcsb.org/download/5YH2.cif -P data/    # save this to the data folder in colab

--2024-01-16 17:31:55--  https://files.rcsb.org/download/5YH2.cif
Resolving files.rcsb.org (files.rcsb.org)... 132.249.213.147
Connecting to files.rcsb.org (files.rcsb.org)|132.249.213.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘data/5YH2.cif’

5YH2.cif                [    <=>             ]   1.39M  1.41MB/s    in 1.0s    

2024-01-16 17:31:56 (1.41 MB/s) - ‘data/5YH2.cif’ saved [1456779]



Load chain C from this CIF file:

In [ ]:
fpath = 'data/5YH2.cif' # .pdb format is also acceptable
chain_id = 'C'
structure = esm.inverse_folding.util.load_structure(fpath, chain_id)
coords, native_seq = esm.inverse_folding.util.extract_coords_from_structure(structure)
print('Native sequence:')
print(native_seq)

Native sequence:
SVLQSLFEHPLYRTVLPDLTEEDTLFNLNAEIRLYPKAASESYPNWLRFHIGINRYELYSRHNPVIAALLRDLLSQKISSVGMKSGGTQLKLIMSFQNYGQALFKPMKQTREQETPPDFFYFSDFERHNAEIAAFHLDRILDFRRVPPVAGRLVNMTREIRDVTRDKKLWRTFFVSPANNICFYGECSYYCSTEHALCGKPDQIEGSLAAFLPDLALAKRKTWRNPWRRSYHKRKKAEWEVDPDYCDEVKQTPPYDRGTRLLDIMDMTIFDFLMGNMDRHHYETFEKFGNDTFIIHLDNGRGFGKHSHDEMSILVPLTQCCRVKRSTYLRLQLLAKEEYKLSSLMEESLLQDRLVPVLIKPHLEALDRRLRLVLKVLSDCVEKDGFSAVVENDLD


Visualize chain C in this CIF file:

In [ ]:
!pip install -q py3Dmol

In [ ]:
try:
    import py3Dmol

    def view_pdb(fpath, chain_id):
        with open(fpath) as ifile:
            system = "".join([x for x in ifile])

        view = py3Dmol.view(width=600, height=400)
        view.addModelsAsFrames(system)
        view.setStyle({'model': -1, 'chain': chain_id}, {"cartoon": {'color': 'spectrum'}})
        view.zoomTo()
        view.show()

except ImportError:
    def view_pdb(fpath, chain_id):
        print("Install py3Dmol to visualize, or use pymol")

Sample sequence and calculate sequence recovery:

## Extract encoder output as structure representation
The encoder output may also be used as a representation for the structure.

For a set of input coordinates with L amino acids, the encoder output will have shape L x 512.

In [ ]:
rep = esm.inverse_folding.util.get_encoder_output(model, alphabet, coords)
len(coords), rep.shape

(395, torch.Size([395, 512]))

In [101]:
! esm-extract esm2_t33_650M_UR50D /content/RBDs_aa_modified.fasta.txt \
  coordoutputRBD.fasta --repr_layers 33 --include mean
! esm-extract esm2_t33_650M_UR50D /content/ACE2_aa_modified.fasta \
  coordoutputACE2.fasta --repr_layers 33 --include mean



Transferred model to GPU
Read /content/RBDs_aa_modified.fasta.txt with 50 sequences
Processing 1 of 3 batches (22 sequences)
Processing 2 of 3 batches (20 sequences)
Processing 3 of 3 batches (8 sequences)
Transferred model to GPU
Read /content/ACE2_aa_modified.fasta with 62 sequences
Processing 1 of 13 batches (5 sequences)
Processing 2 of 13 batches (5 sequences)
Processing 3 of 13 batches (5 sequences)
Processing 4 of 13 batches (5 sequences)
Processing 5 of 13 batches (5 sequences)
Processing 6 of 13 batches (5 sequences)
Processing 7 of 13 batches (5 sequences)
Processing 8 of 13 batches (5 sequences)
Processing 9 of 13 batches (5 sequences)
Processing 10 of 13 batches (5 sequences)
Processing 11 of 13 batches (5 sequences)
Processing 12 of 13 batches (5 sequences)
Processing 13 of 13 batches (2 sequences)


In [108]:
# Specify the folders containing the .pt files
folder_paths = ['/content/coordoutputRBD.fasta', '/content/coordoutputACE2.fasta']

# Flatten the list of filenames
pt_files = [os.path.join(folder, f) for folder in folder_paths for f in os.listdir(folder) if f.endswith('.pt')]

# Iterate over each .pt file
for file_path in pt_files:
    # Load the model using torch.load
    model_dict = torch.load(file_path, map_location=torch.device('cpu'))
    for key, value in model_dict.items():
        print(value)


HKU3-13_GQ153548
{33: tensor([-0.0023,  0.0154, -0.0488,  ..., -0.0725, -0.0469, -0.0348])}
SARS-CoV-1_SZ13_PC03_AY304487
{33: tensor([ 0.0076, -0.0209, -0.0638,  ..., -0.0398, -0.0421, -0.0491])}
Rs4237_KY417147
{33: tensor([ 0.0482,  0.0013, -0.0689,  ..., -0.0579, -0.0728, -0.0399])}
SARS-CoV-1_SZ3_PC03_AY304486
{33: tensor([ 0.0076, -0.0209, -0.0638,  ..., -0.0398, -0.0421, -0.0491])}
SARS-CoV-1_BJ02_HP03M_AY278487
{33: tensor([ 0.0056, -0.0186, -0.0681,  ..., -0.0420, -0.0411, -0.0446])}
RaTG13_MN996532
{33: tensor([ 0.0061, -0.0158, -0.0844,  ..., -0.0339, -0.0621, -0.0375])}
SARS-CoV-1_GZ0402_HP04_AY613947
{33: tensor([-0.0014, -0.0188, -0.0880,  ..., -0.0590, -0.0344, -0.0368])}
Rf1_DQ412042
{33: tensor([ 0.0578, -0.0102, -0.0436,  ..., -0.0115, -0.1090, -0.0348])}
SARS-CoV-1_GD03T0013_HP04_AY525636
{33: tensor([ 0.0019, -0.0249, -0.0739,  ..., -0.0493, -0.0299, -0.0519])}
BtKY72_KY352407
{33: tensor([ 0.0238, -0.0297, -0.0747,  ..., -0.0162, -0.0626, -0.0680])}
SARS-CoV-1_PC4-

In [112]:
# Specify the folders containing the .pt files
folder_paths = ['/content/coordoutputRBD.fasta', '/content/coordoutputACE2.fasta']

formatted_dict = {}

# Iterate over each folder
for folder_path in folder_paths:
    # List all files in the folder with .pt extension
    pt_files = [f for f in os.listdir(folder_path) if f.endswith('.pt')]

    # Iterate over each .pt file in the current folder
    for file_name in pt_files:
        # Construct the full path to the file
        file_path = os.path.join(folder_path, file_name)

        # Load the model using torch.load
        model_dict = torch.load(file_path, map_location=torch.device('cpu'))

        # Extract label and tensor values
        label = model_dict['label']
        tensor_values = model_dict['mean_representations'][33].numpy()

        # Include the first three and last three numbers in the tensor
        first_three = ','.join(map(str, tensor_values[:3]))
        last_three = ','.join(map(str, tensor_values[-3:]))
        formatted_dict[label] = f'{first_three} {last_three}'
print(formatted_dict)

{'HKU3-13_GQ153548': '-0.0022763866,0.0154103255,-0.04875781 -0.07247379,-0.04686134,-0.034843765', 'SARS-CoV-1_SZ13_PC03_AY304487': '0.007573855,-0.020901507,-0.06383968 -0.039771188,-0.04213911,-0.049127687', 'Rs4237_KY417147': '0.048156098,0.0012575239,-0.068928 -0.05791074,-0.07284981,-0.03993152', 'SARS-CoV-1_SZ3_PC03_AY304486': '0.007573855,-0.020901507,-0.06383968 -0.039771188,-0.04213911,-0.049127687', 'SARS-CoV-1_BJ02_HP03M_AY278487': '0.0056334073,-0.0186178,-0.06809834 -0.04202691,-0.04113556,-0.044566154', 'RaTG13_MN996532': '0.006112268,-0.015761022,-0.084432974 -0.03386317,-0.0620671,-0.03750545', 'SARS-CoV-1_GZ0402_HP04_AY613947': '-0.0013685721,-0.018769514,-0.08796831 -0.05899874,-0.034390163,-0.03677005', 'Rf1_DQ412042': '0.057808384,-0.010201075,-0.04362325 -0.011498409,-0.10904997,-0.034847215', 'SARS-CoV-1_GD03T0013_HP04_AY525636': '0.0018673739,-0.024932323,-0.07391755 -0.04929164,-0.029917294,-0.051918983', 'BtKY72_KY352407': '0.023828233,-0.029728387,-0.07467746

In [114]:
formatted_dict[label] = f'{first_three} {last_three}'
for key, value in formatted_dict.items():
    print(f'{key}: {value}')

HKU3-13_GQ153548: -0.0022763866,0.0154103255,-0.04875781 -0.07247379,-0.04686134,-0.034843765
SARS-CoV-1_SZ13_PC03_AY304487: 0.007573855,-0.020901507,-0.06383968 -0.039771188,-0.04213911,-0.049127687
Rs4237_KY417147: 0.048156098,0.0012575239,-0.068928 -0.05791074,-0.07284981,-0.03993152
SARS-CoV-1_SZ3_PC03_AY304486: 0.007573855,-0.020901507,-0.06383968 -0.039771188,-0.04213911,-0.049127687
SARS-CoV-1_BJ02_HP03M_AY278487: 0.0056334073,-0.0186178,-0.06809834 -0.04202691,-0.04113556,-0.044566154
RaTG13_MN996532: 0.006112268,-0.015761022,-0.084432974 -0.03386317,-0.0620671,-0.03750545
SARS-CoV-1_GZ0402_HP04_AY613947: -0.0013685721,-0.018769514,-0.08796831 -0.05899874,-0.034390163,-0.03677005
Rf1_DQ412042: 0.057808384,-0.010201075,-0.04362325 -0.011498409,-0.10904997,-0.034847215
SARS-CoV-1_GD03T0013_HP04_AY525636: 0.0018673739,-0.024932323,-0.07391755 -0.04929164,-0.029917294,-0.051918983
BtKY72_KY352407: 0.023828233,-0.029728387,-0.07467746 -0.016235147,-0.06264966,-0.067961015
SARS-CoV-1